**CSE5525 Lab4**<br>
Author: Xuecheng Liu

**Part 1**<br>
Intuitively, I would like to group funny, super, great, brings in one group since they are all positive. Unfunny would be a group in its own. Epic and great should be one group since they have similar meanings. The grouping is based own my personal understanding to semantic meaning.

Then I will use 'glove-twitter-25' as pretrained word vector. In order to save load time, I will not use the entire embedding, but the result should be similar.

In [80]:
import gensim.downloader
model = gensim.downloader.load('glove-twitter-25') # this step takes quite a while

[==================================================] 100.0% 104.8/104.8MB downloaded


In [81]:
wl = ['funny','unfunny','super','great','epic','brings']
import numpy as np
score = np.zeros((len(wl),len(wl))) # matrix to store the cosine similarity between pairs
for i in range(len(wl)):
  for j in range(len(wl)):
    score[i,j] = model.similarity(wl[i],wl[j])

In [82]:
score

array([[1.        , 0.55263644, 0.62186861, 0.73716587, 0.74991006,
        0.66038907],
       [0.55263644, 1.        , 0.1259675 , 0.26232597, 0.34147432,
        0.28290182],
       [0.62186861, 0.1259675 , 1.        , 0.69745016, 0.71294999,
        0.39232939],
       [0.73716587, 0.26232597, 0.69745016, 0.99999994, 0.84397268,
        0.87443566],
       [0.74991006, 0.34147432, 0.71294999, 0.84397268, 1.        ,
        0.74569476],
       [0.66038907, 0.28290182, 0.39232939, 0.87443566, 0.74569476,
        1.        ]])

For experimental purpose, I choose a different embedding.

In [83]:
model2 = gensim.downloader.load('glove-wiki-gigaword-50')


[==================================================] 100.0% 66.0/66.0MB downloaded


In [94]:
wl = ['funny','unfunny','super','great','epic','brings']
import numpy as np
score2 = np.zeros((len(wl),len(wl))) # matrix to store the cosine similarity between pairs
for i in range(len(wl)):
  for j in range(len(wl)):
    score2[i,j] = model2.similarity(wl[i],wl[j])
score2

array([[ 1.        ,  0.66310102,  0.30789351,  0.44568205,  0.40846473,
         0.50398821],
       [ 0.66310102,  0.99999994, -0.0874329 ,  0.00198272,  0.26645535,
         0.11681217],
       [ 0.30789351, -0.0874329 ,  1.        ,  0.45516908,  0.32621685,
         0.34351856],
       [ 0.44568205,  0.00198272,  0.45516908,  0.99999994,  0.56852591,
         0.74064589],
       [ 0.40846473,  0.26645535,  0.32621685,  0.56852591,  1.        ,
         0.51670569],
       [ 0.50398821,  0.11681217,  0.34351856,  0.74064589,  0.51670569,
         0.99999994]])

As you can see, the result differs among different pretrained word vectors and my intuition. The reason for this might be issue of semantic ambiguity in different context. One takeway might be we need to choose a proper training set to deal with real word problems.

**Part 2 option 2**

I will use Cornell's movie review and I have attached it within the submission.<br>
The first step is to preprocess the data and store reviews and class in one dataframe.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
!unzip mix20_rand700_tokens_0211.zip

In [3]:
cd /content/mix20_rand700_tokens_0211

/content/mix20_rand700_tokens_0211


In [4]:
cd tokens

/content/mix20_rand700_tokens_0211/tokens


In [5]:
import os
all_files = os.listdir("neg/") # all negative files

['cv168_tok-4250.txt', 'cv694_tok-19302.txt', 'cv604_tok-21195.txt', 'cv661_tok-19547.txt', 'cv169_tok-10229.txt', 'cv267_tok-13643.txt', 'cv156_tok-19046.txt', 'cv348_tok-22403.txt', 'cv183_tok-18186.txt', 'cv004_tok-25944.txt', 'cv527_tok-28453.txt', 'cv069_tok-11865.txt', 'cv257_tok-20349.txt', 'cv149_tok-14540.txt', 'cv307_tok-20139.txt', 'cv419_tok-16024.txt', 'cv514_tok-23401.txt', 'cv207_tok-29808.txt', 'cv247_tok-14944.txt', 'cv671_tok-2156.txt', 'cv248_tok-17823.txt', 'cv668_tok-23578.txt', 'cv585_tok-8500.txt', 'cv011_tok-7845.txt', 'cv519_tok-12701.txt', 'cv114_tok-16135.txt', 'cv295_tok-13516.txt', 'cv139_tok-22512.txt', 'cv586_tok-19488.txt', 'cv614_tok-15689.txt', 'cv151_tok-26106.txt', 'cv558_tok-29006.txt', 'cv478_tok-10537.txt', 'cv693_tok-16529.txt', 'cv682_tok-16618.txt', 'cv590_tok-7983.txt', 'cv034_tok-25395.txt', 'cv161_tok-23503.txt', 'cv283_tok-24782.txt', 'cv453_tok-17410.txt', 'cv298_tok-29444.txt', 'cv599_tok-11859.txt', 'cv285_tok-10266.txt', 'cv543_tok-1028

In [10]:
cd neg/

/content/mix20_rand700_tokens_0211/tokens/neg


In [41]:
data = [] # list of list used to create data frame
for name in all_files:
  with open(name, 'rb') as f:
    content = f.read().decode('utf8','ignore')
    data.append([content,-1])


In [49]:
cd /content/mix20_rand700_tokens_0211/tokens

/content/mix20_rand700_tokens_0211/tokens


In [ ]:
all_pos = os.listdir('pos/')

In [51]:
cd pos/

/content/mix20_rand700_tokens_0211/tokens/pos


In [52]:
for name in all_pos:
  with open(name, 'rb') as f:
    content = f.read().decode('utf8','ignore')
    data.append([content,1])

In [55]:
df = pd.DataFrame(data, columns = ['review', 'label'])

Up to now, we have all reviews and their label stored in df variable

In [60]:
# wordDict maps words to id
wordDict = {}
idCounter = 0
for i in range(df.shape[0]):
  allWords = df.iloc[i,0].split(" ")
  for word in allWords:
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1
idCounter

43308

In [61]:
dict(list(wordDict.items())[0:10])

{'comes': 6,
 'imaginative': 2,
 'its': 8,
 'mallrats': 5,
 'most': 1,
 'of': 4,
 'opening': 9,
 'part': 3,
 'the': 0,
 'with': 7}

In [65]:
X = np.zeros((df.shape[0], idCounter),dtype='float')
for tweetIdx in range(df.shape[0]):
  allWords = df.iloc[tweetIdx,0].split(" ")
  for word in allWords:
    wordId=wordDict[word]
    X[tweetIdx, wordId]  = 1

In [66]:
np.sum(X[0:5, ], axis = 1)

array([358., 228., 370., 277., 266.])

In [69]:
y = np.array(df.iloc[:,1])
y[0:5]

array([-1, -1, -1, -1, -1])

In [70]:
numNeg = np.sum(y < 0)
numPos = np.sum(y > 0) #len(y) - numNeg
probNeg = numNeg / (numNeg + numPos)
probPos = 1 - probNeg
display(numNeg, numPos, probNeg, probPos)

692

694

0.49927849927849927

0.5007215007215007

In [71]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 0)
display(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)

(1108, 43308)

(278, 43308)

(1108,)

(278,)

In [72]:
def compute_distros(x,y,weights=None):
  #
  # probWordGivenPositive: P(word|Sentiment = +ive)
  probWordGivenPositive=np.sum(x[y>=0,:],axis=0) #Sum each word (column) to count how many times each word shows up (in positive examples)
  probWordGivenPositive=probWordGivenPositive/np.sum(y>=0) #Divide by total number of (positive) examples to give distribution

  # probWordGivenNegative: P(word|Sentiment = -ive)
  probWordGivenNegative=np.sum(x[y<0,:],axis=0)
  probWordGivenNegative=probWordGivenNegative/np.sum(y<0)

  # priorPositive: P(Sentiment = +ive)
  priorPositive = np.sum(y>=0)/y.shape[0] #Number of positive examples vs. all examples
  # priorNegative: P(Sentiment = -ive)
  priorNegative = 1 - priorPositive
  #  (note these last two form one distribution)

  return probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative

# compute distributions here
probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain)

In [73]:
def compute_logdistros(distros, min_prob):
  if True:
    #Assume missing words are simply very rare
    #So, assign minimum probability to very small elements (e.g. 0 elements)
    distros=np.where(distros>=min_prob,distros,min_prob)
    #Also need to consider minimum probability for "not" distribution
    distros=np.where(distros<=(1-min_prob),distros,1-min_prob)

    return  np.log(distros), np.log(1-distros)
  else:
    #Ignore missing words (assume they have P==1)
    return np.log(np.where(distros>0,distros,1)), np.log(np.where(distros<1,1-distros,1))

min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

In [75]:
def classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
               logPriorPositive, logPriorNegative, ignore_absent_words=False):
  logProbPositiveGivenWords = logPriorPositive + np.sum(np.multiply(words,logProbWordPresentGivenPositive))
  logProbNegativeGivenWords = logPriorNegative + np.sum(np.multiply(words,logProbWordPresentGivenNegative))
  #Same trick as above, 1-words makes present words 0, masking them
  logProbPositiveGivenWords += np.sum(np.multiply(1-words,logProbWordAbsentGivenPositive))
  logProbNegativeGivenWords += np.sum(np.multiply(1-words,logProbWordAbsentGivenNegative))

  label = 1 #Assume classification is "1" and calculate confidence accordingly
  confidence = logProbPositiveGivenWords - logProbNegativeGivenWords
  if confidence < 0: #If confidence is negative, our assumption was wrong, so fix it
    label = -1
    confidence = -confidence
  #GRADER NOTE: Students were not _required_ to calculate confidence

  return label, confidence
  
print(classifyNB(xTest[100, ], logProbWordPresentGivenPositive,logProbWordAbsentGivenPositive,
                               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
                               logPriorPositive, logPriorNegative))

(-1, 33.451693506629454)


In [76]:
def testNB(xTest, yTest, 
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
           logPriorPositive, logPriorNegative, ignore_absent_words=False):

  # compute avgErr
  accuracy=0
  for i in range(yTest.shape[0]):
    label, confidence = classifyNB(xTest[i, ], logProbWordPresentGivenPositive,logProbWordAbsentGivenPositive,
                                               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
                                               logPriorPositive, logPriorNegative,
                                               ignore_absent_words=ignore_absent_words)
    if label == yTest[i]:
      accuracy+=1

  accuracy/=yTest.shape[0]
  print("Accuracy of NB is", accuracy)
  return accuracy

testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative)

testNB(xTrain, yTrain, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative)

Accuracy of NB is 0.802158273381295
Accuracy of NB is 0.9819494584837545


0.9819494584837545

In [77]:
# 10 train/test splits
def do_trials(trialCnt,X,y,weights=None, ignore_absent_words=False):
  accuracies=[]
  print("Trials:\n-------")
  for trial in range(trialCnt):
    if weights is not None:
      xTrain, xTest, yTrain, yTest, wTrain, wTest = train_test_split(X, y, weights, test_size = 0.2)
    else:
      xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
      wTrain=None

    probWordGivenPositive, probWordGivenNegative, priorPositive, priorNegative = compute_distros(xTrain,yTrain,weights=wTrain)

    min_prob = 1/yTrain.shape[0] #Assume very rare words only appeared once
    logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive = compute_logdistros(probWordGivenPositive,min_prob)
    logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative = compute_logdistros(probWordGivenNegative,min_prob)
    logPriorPositive, logPriorNegative = compute_logdistros(priorPositive,min_prob)

    accuracy = testNB(xTest, yTest, 
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive, 
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative, 
       logPriorPositive, logPriorNegative, ignore_absent_words=ignore_absent_words)
    
    accuracies.append(accuracy)

  accuracies=np.array(accuracies)
  acc_mean = np.mean(accuracies)
  acc_stddev = np.std(accuracies)
  print("-------\nMean accuracy =",acc_mean,"\nStd dev = ",acc_stddev)

do_trials(10,X,y)

Trials:
-------
Accuracy of NB is 0.7841726618705036
Accuracy of NB is 0.7338129496402878
Accuracy of NB is 0.7769784172661871
Accuracy of NB is 0.7733812949640287
Accuracy of NB is 0.8237410071942446
Accuracy of NB is 0.8057553956834532
Accuracy of NB is 0.7482014388489209
Accuracy of NB is 0.8057553956834532
Accuracy of NB is 0.8057553956834532
Accuracy of NB is 0.7877697841726619
-------
Mean accuracy = 0.7845323741007195 
Std dev =  0.0264064059744359


Model Comparison from sklearn

In [78]:
from sklearn.naive_bayes import BernoulliNB

In [79]:
# 10 train/test splits using sklearn
def try_sklean_nb(trialCnt,X,y):
  accuracies=[]
  print("Trials:\n-------")
  for trial in range(trialCnt):
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)
    
    clf = BernoulliNB()
    clf.fit(xTrain, yTrain)

    labels=clf.predict(xTest)
    accuracy=np.sum(labels==yTest)/yTest.shape[0]
    print(accuracy)

    accuracies.append(accuracy)

  accuracies=np.array(accuracies)
  acc_mean = np.mean(accuracies)
  acc_stddev = np.std(accuracies)
  print("-------\nMean accuracy =",acc_mean,"\nStd dev = ",acc_stddev)

try_sklean_nb(10,X,y)

Trials:
-------
0.7877697841726619
0.7949640287769785
0.8129496402877698
0.7949640287769785
0.7482014388489209
0.7769784172661871
0.7589928057553957
0.7553956834532374
0.7697841726618705
0.7230215827338129
-------
Mean accuracy = 0.7723021582733812 
Std dev =  0.025336103097755903
